In [1]:
from util import *

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('../vizwiz_skill_typ_train.csv', skipinitialspace = True, engine='python')
df_val = pd.read_csv('../vizwiz_skill_typ_val.csv', skipinitialspace = True, engine='python')
df_train_features = pd.read_csv('../vizwiz_features_train.csv', skipinitialspace = True, engine='python')
df_val_features = pd.read_csv('../vizwiz_features_val.csv', skipinitialspace = True, engine='python')
nn_train = pd.read_csv("nn_features_train.csv")
api_train = pd.read_csv("api_features_train.csv")

In [5]:
nn_train.head(2)

,qid,vgg_tags alexnet_tags,txt,col
0,VizWiz_train_000000000000.jpg,perfume essence packet,1,0
1,VizWiz_train_000000000001.jpg,espresso maker lighter light igniter ignitor,1,0


In [35]:
api_train.head(2)

,qid,adult,categories,descriptions,tags
0,VizWiz_train_000000000000.jpg,False,others_,a bottle of wine on a table,'bottle' 'indoor' 'table' 'sitting' 'wine'...
1,VizWiz_train_000000000001.jpg,False,others_,a kitchen with a sink and a microwave,'indoor' 'cabinet' 'kitchen' 'small' 'refr...


In [13]:
nn_train.set_index('qid', inplace=True)
api_train.set_index('qid', inplace=True)

In [41]:
api_train.isnull().any()

adult           False
categories      False
descriptions    False
tags            False
dtype: bool

In [40]:
nn_train.isnull().any()

vgg_tags alexnet_tags    False
txt                      False
col                      False
dtype: bool

In [3]:
df_train = df_train[['QID', 'TXT', 'COL']]

In [4]:
# use metadata for prediction
df_train_features = df_train_features[['QID', 'I_NUM_CATEGORIES', 'I_NUM_TAGS', 'I_NUM_COLOURS', 'I_NUM_FACES']]
df_train_features.set_index("QID", inplace=True)

In [5]:
train = df_train.join(df_train_features, on="QID")

In [6]:
df_val = df_val[['QID', 'TXT', 'COL']]
df_val_features = df_val_features[['QID', 'I_NUM_CATEGORIES', 'I_NUM_TAGS', 'I_NUM_COLOURS', 'I_NUM_FACES']]
df_val_features.set_index("QID", inplace=True)

In [7]:
val = df_val.join(df_val_features, on='QID')

In [8]:
# use validation as test set; split training set into train vs validaiton set
# text category
X_train = train[['I_NUM_CATEGORIES', 'I_NUM_TAGS', "I_NUM_COLOURS", "I_NUM_FACES"]].values
X_test = val[['I_NUM_CATEGORIES', 'I_NUM_TAGS', "I_NUM_COLOURS", "I_NUM_FACES"]].values
Y_train_txt = train['TXT'].values.ravel()
Y_test_txt = val['TXT'].values.ravel()
Y_train_col = train['COL'].values.ravel()
Y_test_col = val['COL'].values.ravel()
#X_train, X_test, Y_train, Y_test = train_test_split(train[['I_NUM_CATEGORIES', 'I_NUM_TAGS', "I_NUM_COLOURS", "I_NUM_FACES"]].values, 
#                                                    train[['TXT']].values, random_state=42, stratify=train[['TXT']].values)

In [9]:
logit(X_train, X_test, Y_train_txt, Y_test_txt)

10-fold cross-validated score on training set: 0.5806465968155577
Best C = 5.0
Best score on test set: 0.6454908929364727
AUC:  0.6212849048143165


In [10]:
logit(X_train, X_test, Y_train_col, Y_test_col)

10-fold cross-validated score on training set: 0.4673249810683796
Best C = 0.5
Best score on test set: 0.6406041759218125
AUC:  0.5498170171537006


In [11]:
majority_vote_tuning(X_train, X_test, Y_train_txt, Y_test_txt)

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

10-fold cross validation score on training set: 0.6399196896472267
Best score on test set: 0.6406041759218125
AUC:  0.596213362683951


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
knn(X_train, X_test, Y_train_txt, Y_test_txt)

10-fold cross-validated score on training set: 0.6285454623991052
Best K = 120
Best score on test set: 0.6370501999111506
AUC:  0.6128127675186499


In [13]:
knn(X_train, X_test, Y_train_col, Y_test_col)

10-fold cross-validated score on training set: 0.6155398558463381
Best K = 120
Best score on test set: 0.6321634828964905
AUC:  0.5427733907633404


In [14]:
svm(X_train, X_test, Y_train_txt, Y_test_txt)

Best score on test set: 0.6490448689471346
AUC:  0.6112241653418125


In [15]:
svm(X_train, X_test, Y_train_col, Y_test_col)

Best score on test set: 0.6188360728565082
AUC:  0.5


In [16]:
lda(X_train, X_test, Y_train_txt, Y_test_txt)

10-fold cross validation score on training set: 0.6425113927458304
Best score on test set: 0.6450466459351399
AUC:  0.6199270311035017


In [ ]:
lda(X_train, X_test, Y_train_col, Y_test_col)